In [13]:
#!/usr/bin/python3

# Webserver dependencies
from http.server import BaseHTTPRequestHandler, HTTPServer
import argparse
import time
import google.cloud.storage as storage
import google.cloud.pubsub as pubsub

# Sql dependencies
import os
from google.cloud.sql.connector import Connector, IPTypes
import pymysql
import socket, struct
import sqlalchemy

import numpy as np
import pandas as pd

from sklearn import linear_model
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [2]:
PROJECT_ID = "feisty-gasket-398719"
TOPIC_ID = "my-topic"
SUBSCRIPTION_NAME = "my-topic-sub"
INSTANCE_CONNECTION_NAME = "feisty-gasket-398719:us-east1:instance-tigeryi"
DB_USER = "root"
DB_PASS = ""
DB_NAME = "dbhw5"
DB_PRIVATE_IP = False
BANNED_COUNTRIES = ["North Korea", "Iran", "Cuba", "Myanmar", "Iraq", "Libya", "Sudan", "Zimbabwe", "Syria"]
HTTP_METHODS = ['GET', 'HEAD', 'POST', 'PUT', 'DELETE', 'CONNECT', 'OPTIONS', 'TRACE', 'PATCH']

In [3]:
def connect_with_connector() -> sqlalchemy.engine.base.Engine:
        """
        Initializes a connection pool for a Cloud SQL instance of MySQL.
        
        Uses the Cloud SQL Python Connector package.
        """
        # Note: Saving credentials in environment variables is convenient, but not
        # secure - consider a more secure solution such as
        # Cloud Secret Manager (https://cloud.google.com/secret-manager) to help
        # keep secrets safe.

        instance_connection_name = INSTANCE_CONNECTION_NAME # os.environ["INSTANCE_CONNECTION_NAME"]  # e.g. 'project:region:instance'
        db_user = DB_USER # os.environ["DB_USER"]  # e.g. 'my-db-user'
        db_pass = DB_PASS # os.environ["DB_PASS"]  # e.g. 'my-db-password'
        db_name = DB_NAME # os.environ["DB_NAME"]  # e.g. 'my-database'

        ip_type = IPTypes.PRIVATE if DB_PRIVATE_IP else IPTypes.PUBLIC # if os.environ.get("DB_PRIVATE_IP")

        connector = Connector(ip_type)

        def getconn() -> pymysql.connections.Connection:
            conn: pymysql.connections.Connection = connector.connect(
                instance_connection_name,
                "pymysql",
                user=db_user,
                password=db_pass,
                db=db_name,
            )
            return conn
        
        pool = sqlalchemy.create_engine(
            "mysql+pymysql://",
            creator=getconn,
            # ...
        )
        return pool

In [4]:
pool = connect_with_connector()

In [5]:
query = '''select country2, gender2, age2, income2, country, gender, age, income from table2;'''


In [6]:
df = pd.read_sql_query(query, pool)

In [7]:
print(df.head())

,country2,gender2,age2,income2,country,gender,age,income
0,142,0,5,4,Romania,Male,56-65,60k-100k
1,180,0,5,7,Turkey,Male,56-65,250k+
2,15,0,1,0,Belarus,Male,17-25,0-10k
3,82,1,1,6,Italy,Female,17-25,150k-250k
4,81,0,1,2,Israel,Male,17-25,20k-40k


In [8]:
X = df[['country2', 'gender2', 'age2']]
X

In [9]:
y = df['income2']
y

In [10]:
clf = DecisionTreeClassifier()

In [11]:
clf.fit(X,y)

DecisionTreeClassifier()

In [12]:
clf.score(X,y)

1.0

In [ ]:
y_pred = clf.predict(X)

In [ ]:
accuracy_score(y, y_pred)